# Notebook 04: SHAP Values

## Unpacking Predictions

SHAP answers: How much did each feature contribute to this prediction? Based on game theory, ensuring local attributions sum to prediction minus baseline.

In [ ]:
import shap
import xgboost as xgb
from sklearn.linear_model import Ridge
from src.utils import set_seed
set_seed(42)

In [ ]:
# === TODO: Fit Ridge, compute SHAP with LinearExplainer on 500-row sample
# Acceptance: SHAP summary plot; 2-sentence interpretation

In [ ]:
# === TODO: Fit XGBoost, compute SHAP with TreeExplainer
# Acceptance: SHAP beeswarm plot; note top 3 features

In [ ]:
# === TODO: Compare SHAP ranking vs permutation importance
# Acceptance: Table with top 5 features by each method